In [1]:
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import json

In [2]:
train = pd.read_csv('train_Y.csv')
submit = pd.read_csv('sample_submit.csv')

In [3]:
train_user = train['user']
test_user = submit['user']
y_train = train['label']

In [4]:
total_user = pd.concat([train_user, test_user])
total_user = total_user.sort_values().reset_index(drop=True)

In [5]:
sleep_list = sorted(glob('StduentLife_EMA/EMA/response/Sleep/*.json'))
social_list = sorted(glob('StduentLife_EMA/EMA/response/Social/*.json'))
activity_list = sorted(glob('StduentLife_EMA/EMA/response/Activity/*.json'))

In [6]:
sleep = np.array([sleep for sleep in sleep_list for user in total_user if user in sleep])
social = np.array([social for social in social_list for user in total_user if user in social])
activity = np.array([activity for activity in activity_list for user in total_user if user in activity])

In [7]:
people_hour = []
people_rate = []

for data in tqdm(sleep) : 
    sleep_data = json.load(open(data))
    
    people_hour.append(['NaN' if 'hour' not in res else int(res['hour']) * 0.5 + 2.5 for res in sleep_data])
    people_rate.append(['NaN' if 'rate' not in res else ((int(res['rate']) - 5) * -1) for res in sleep_data])

100%|██████████| 46/46 [00:00<00:00, 13881.43it/s]


In [8]:
people_contact = []
social_answer_set = ['NaN', 0, 5, 10, 20, 50, 100]

for data in tqdm(social):
    social_data = json.load(open(data))
    
    people_contact.append([social_answer_set[0 if 'number' not in res else int(res['number'])] for res in social_data])

100%|██████████| 46/46 [00:00<00:00, 15338.10it/s]


In [9]:
people_answer_set = ['NaN', 0, 0.11, 0.26, 0.51, 0.76]

people_work_alone = [] 
people_work_other = []
people_relaxing_alone = [] 
people_relaxing_other = []

for act in tqdm(activity):
    activity_data = json.load(open(act))
    
    people_work_alone.append([people_answer_set[0 if 'working' not in res else int(res['working'])] for res in activity_data])
    people_work_other.append([people_answer_set[0 if 'other_working' not in res else int(res['other_working'])] for res in activity_data])
    people_relaxing_alone.append([people_answer_set[0 if 'relaxing' not in res else int(res['relaxing'])] for res in activity_data])
    people_relaxing_other.append([people_answer_set[0 if 'other_relaxing' not in res else int(res['other_relaxing'])] for res in activity_data])

100%|██████████| 46/46 [00:00<00:00, 18523.23it/s]


In [20]:
student_sleep = pd.concat([total_user, pd.DataFrame(people_hour).astype('float64')], axis=1)
student_sleep_rate = pd.concat([total_user, pd.DataFrame(people_rate).astype('float64')], axis=1)
student_contact = pd.concat([total_user, pd.DataFrame(people_contact).astype('float64')], axis=1)
student_working_alone = pd.concat([total_user, pd.DataFrame(people_work_alone).astype('float64')], axis=1)
student_working_other = pd.concat([total_user, pd.DataFrame(people_work_other).astype('float64')], axis=1)
student_relaxing_alone = pd.concat([total_user, pd.DataFrame(people_relaxing_alone).astype('float64')], axis=1)
student_relaxing_other = pd.concat([total_user, pd.DataFrame(people_relaxing_other).astype('float64')], axis=1)

In [31]:
student_sleep[student_sleep['user'].isin(train_user)].iloc[:,1:].T.describe()

,0,1,3,5,6,7,9,12,14,17,...,24,26,33,34,35,36,37,39,43,44
count,55.000000,27.000000,28.000000,6.000000,24.000000,41.000000,47.000000,30.000000,65.000000,55.000000,...,11.000000,41.000000,58.000000,11.000000,23.000000,9.000000,41.000000,34.000000,61.000000,58.000000
mean,6.054545,6.111111,6.535714,6.250000,6.020833,6.000000,6.393617,5.983333,6.453846,6.418182,...,5.227273,5.926829,7.034483,6.363636,5.630435,5.777778,5.902439,6.029412,6.245902,6.129310
std,1.508227,0.609750,1.070899,0.689202,0.541318,1.423025,0.589179,0.636306,1.070642,1.169693,...,1.009050,1.693963,1.127262,0.452267,0.741886,1.301708,1.817345,1.236694,1.293003,0.881388
min,3.000000,5.000000,4.500000,5.000000,5.000000,2.500000,5.000000,4.500000,3.000000,3.000000,...,3.500000,2.500000,5.500000,5.500000,4.000000,3.000000,3.500000,3.000000,3.000000,4.500000
25%,4.750000,6.000000,6.000000,6.125000,5.500000,5.500000,6.000000,5.500000,6.000000,6.000000,...,4.750000,5.000000,6.500000,6.500000,5.500000,5.500000,4.500000,5.500000,5.500000,5.500000
50%,6.500000,6.000000,6.250000,6.500000,6.000000,6.000000,6.500000,6.000000,6.500000,6.500000,...,5.000000,6.000000,6.500000,6.500000,5.500000,6.000000,5.500000,6.000000,6.000000,6.000000
75%,7.000000,6.500000,7.000000,6.500000,6.500000,6.500000,6.500000,6.500000,7.000000,6.750000,...,6.000000,6.500000,8.000000,6.500000,6.000000,6.500000,7.000000,6.500000,7.000000,6.500000
max,10.000000,7.500000,10.000000,7.000000,7.000000,10.000000,7.500000,7.000000,9.000000,9.500000,...,6.500000,10.500000,11.000000,7.000000,7.000000,7.000000,10.000000,9.000000,11.000000,9.500000


In [21]:
train_features_student_sleep = student_sleep[student_sleep['user'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_sleep_rate = student_sleep_rate[student_sleep_rate['user'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_contact = student_contact[student_contact['user'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_working_alone = student_working_alone[student_working_alone['user'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_working_other = student_working_other[student_working_other['user'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_relaxing_alone = student_relaxing_alone[student_relaxing_alone['user'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()
train_features_student_relaxing_other = student_relaxing_other[student_relaxing_other['user'].isin(train_user)].iloc[:,1:].T.describe().to_numpy()

In [12]:
test_features_student_sleep = student_sleep[student_sleep['user'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_sleep_rate = student_sleep_rate[student_sleep_rate['user'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_contact = student_contact[student_contact['user'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_working_alone = student_working_alone[student_working_alone['user'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_working_other = student_working_other[student_working_other['user'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_relaxing_alone = student_relaxing_alone[student_relaxing_alone['user'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()
test_features_student_relaxing_other = student_relaxing_other[student_relaxing_other['user'].isin(test_user)].iloc[:,1:].T.describe().to_numpy()

In [13]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

imp_mean.fit(train_features_student_sleep)
train_features_student_sleep = imp_mean.transform(train_features_student_sleep)
test_features_student_sleep = imp_mean.transform(test_features_student_sleep)

imp_mean.fit(train_features_student_sleep_rate)
train_features_student_sleep_rate = imp_mean.transform(train_features_student_sleep_rate)
test_features_student_sleep_rate = imp_mean.transform(test_features_student_sleep_rate)

imp_mean.fit(train_features_student_contact)
train_features_student_contact = imp_mean.transform(train_features_student_contact)
test_features_student_contact = imp_mean.transform(test_features_student_contact)

imp_mean.fit(train_features_student_working_alone)
train_features_student_working_alone = imp_mean.transform(train_features_student_working_alone)
test_features_student_working_alone = imp_mean.transform(test_features_student_working_alone)

imp_mean.fit(train_features_student_working_other)
train_features_student_working_other = imp_mean.transform(train_features_student_working_other)
test_features_student_working_other = imp_mean.transform(test_features_student_working_other)

imp_mean.fit(train_features_student_relaxing_alone)
train_features_student_relaxing_alone = imp_mean.transform(train_features_student_relaxing_alone)
test_features_student_relaxing_alone = imp_mean.transform(test_features_student_relaxing_alone)

imp_mean.fit(train_features_student_relaxing_other)
train_features_student_relaxing_other = imp_mean.transform(train_features_student_relaxing_other)
test_features_student_relaxing_other = imp_mean.transform(test_features_student_relaxing_other)

In [14]:
train_features = [train_features_student_sleep,
train_features_student_sleep_rate,
train_features_student_contact,
train_features_student_working_alone,
train_features_student_working_other,
train_features_student_relaxing_alone,
train_features_student_relaxing_other]

test_features = [test_features_student_sleep,
test_features_student_sleep_rate,
test_features_student_contact,
test_features_student_working_alone,
test_features_student_working_other,
test_features_student_relaxing_alone,
test_features_student_relaxing_other
]

In [15]:
X_train = []

for feature in train_features:
    for i in range(feature.shape[1]) : 
        tmp_x = []
        tmp_x.append(feature[1:8, i])
        X_train.append(tmp_x)

X_train = np.array(X_train)
X_train = X_train.reshape(train_features_student_sleep.shape[1],-1)

X_test = []

for feature in test_features:
    for i in range(feature.shape[1]) : 
        tmp_x = []
        tmp_x.append(feature[1:8, i])
        X_test.append(tmp_x)

X_test = np.array(X_test)
X_test = X_test.reshape(test_features_student_sleep.shape[1],-1)

In [16]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = [{'C': [1.0, 10.0, 100.0, 1000.0], 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']}]

model = GridSearchCV(estimator=SVC(), param_grid=param_grid)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
submit['label'] = y_pred
submit.to_csv("submit.csv", index=False)